# Automated ML Training

In [ ]:
%pip install autokeras==1.0.19

In [1]:
%cd stock-pred

/mlsteam/lab/stock-pred


In [2]:
# Global settings
WINDOW = 20
EPOCHS = 15
BATCH = 20
TEST_RATIO = .2

DATA_PATH = '/mlsteam/data/stock_prices/20220512_tesla.pkl'
SCALER_PATH = '/lab/trained/scaler.pkl'
MODEL_PATH = '/lab/trained/model'
TENSORBOARD_PATH = '/tensorboard'

## Load Data

In [3]:
from train import load_data

_, stock_prices = load_data(DATA_PATH)
x_all = stock_prices[:-1, None]
y_all = stock_prices[1:]

train_split = int(len(y_all) * (1.0 - TEST_RATIO))
x_train, y_train = x_all[:train_split], y_all[:train_split]
x_val, y_val = x_all[train_split:], y_all[train_split:]
x_test, y_test = x_all[train_split:], y_all[train_split:]

In [4]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(201, 1) (201,)
(51, 1) (51,)


## Train Model

In [5]:
import autokeras as ak

clf = ak.TimeseriesForecaster(
    column_names=['stock_price'],
    lookback=WINDOW,
    predict_from=1,
    predict_until=1,
    project_name='stock-pred-automl',
    max_trials=20,
    objective='val_loss'
)
clf.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_val, y_val),
    batch_size=BATCH,
    epochs=EPOCHS
)

Trial 20 Complete [00h 00m 13s]
val_loss: 20220.490234375

Best val_loss So Far: 20219.083984375
Total elapsed time: 00h 03m 29s
INFO:tensorflow:Oracle triggered exit
Epoch 1/15
10/10 [==============================] - 5s 120ms/step - loss: 522117.3750 - mean_squared_error: 522117.3750 - val_loss: 354193.3750 - val_mean_squared_error: 354193.3750
Epoch 2/15
10/10 [==============================] - 0s 14ms/step - loss: 168714.0469 - mean_squared_error: 168714.0469 - val_loss: 141274.0312 - val_mean_squared_error: 141274.0312
Epoch 3/15
10/10 [==============================] - 0s 15ms/step - loss: 67784.7656 - mean_squared_error: 67784.7656 - val_loss: 68349.1016 - val_mean_squared_error: 68349.1016
Epoch 4/15
10/10 [==============================] - 0s 17ms/step - loss: 40051.7461 - mean_squared_error: 40051.7461 - val_loss: 41184.4180 - val_mean_squared_error: 41184.4180
Epoch 5/15
10/10 [==============================] - 0s 13ms/step - loss: 32839.5742 - mean_squared_error: 32839.5742

INFO:tensorflow:Assets written to: ./stock-pred-automl/best_model/assets


INFO:tensorflow:Assets written to: ./stock-pred-automl/best_model/assets


## Test Model

In [12]:
from train import get_rmse, get_mape

y_predict = clf.predict(x_test)  # FIXME: don't know why I cannot use x_test here

rmse = get_rmse(pred=y_predict, actual=y_train)
mape = get_mape(pred=y_predict, actual=y_train)
print(f'RMSE: {rmse}, MAPE: {mape}')

ValueError: The prediction data requires the original training data to make predictions on subsequent data points